In [ ]:
# 💾 Guardar resultados en Google Drive

# Verificar que los pasos anteriores se ejecutaron
if 'results' not in globals():
    print("❌ ERROR: La variable 'results' no está definida.")
    print("📋 Por favor ejecuta primero:")
    print("   1. Todas las celdas de configuración (en orden)")
    print("   2. La celda de 'Ejecutar análisis acumulativo'")
    print("   3. Luego ejecuta esta celda para guardar")
    print("\n💡 TIP: Usa Runtime → Run all para ejecutar todo en orden")
else:
    def save_results_to_drive(results, filename: str) -> bool:
        """Guardar resultados en Google Drive."""
        print(f"💾 Guardando resultados: {filename}")
        
        try:
            # Convertir a JSON
            json_str = json.dumps(results, indent=2, ensure_ascii=False)
            file_io = io.BytesIO(json_str.encode('utf-8'))
            
            # Metadata del archivo
            file_metadata = {
                'name': filename,
                'parents': []  # Raíz de Drive
            }
            
            # Subir archivo
            media = MediaIoBaseUpload(file_io, mimetype='application/json')
            file = service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id'
            ).execute()
            
            print(f"✅ Resultados guardados con ID: {file.get('id')}")
            return True
            
        except Exception as e:
            print(f"❌ Error guardando resultados: {e}")
            return False

    # Guardar resultados
    results_filename = config['output_config']['results_filename']
    
    success = save_results_to_drive(results, results_filename)
    
    if success:
        print(f"\n🎉 ¡Análisis N Preguntas Completado Exitosamente!")
        print(f"📄 Archivo de resultados: {results_filename}")
        print(f"📊 Preguntas procesadas: {results['execution_stats']['questions_processed']}")
        print(f"🤖 Modelos evaluados: {results['execution_stats']['models_evaluated']}")
        print(f"⏱️ Tiempo total: {results['execution_stats']['total_time']:.1f} segundos")
        
        print(f"\n📋 Próximos pasos:")
        print(f"1. Ve a la aplicación Streamlit")
        print(f"2. Navega a 'Resultados Análisis N Preguntas'")
        print(f"3. Busca el archivo: {results_filename}")
        print(f"4. Explora las visualizaciones y métricas detalladas")
        
    else:
        print("❌ Error al guardar resultados en Google Drive")
        print("💡 Los resultados están disponibles en la variable 'results'")

In [ ]:
# 🚀 Ejecutar análisis acumulativo

# Verificar que los pasos anteriores se ejecutaron
if 'config' not in globals():
    print("❌ ERROR: La variable 'config' no está definida.")
    print("📋 Por favor ejecuta las celdas en orden:")
    print("   1. Instalación de dependencias")
    print("   2. Importaciones")
    print("   3. Autenticación con Google Drive")
    print("   4. Cargar configuración")
    print("   5. Conectar a ChromaDB")
    print("   6. Cargar modelos")
    print("   7. Esta celda (Ejecutar análisis)")
    raise NameError("Ejecuta las celdas anteriores primero")

if 'questions' not in globals():
    print("❌ ERROR: La variable 'questions' no está definida.")
    print("📋 Ejecuta la celda de ChromaDB primero")
    raise NameError("Ejecuta la celda de ChromaDB primero")

if 'models' not in globals():
    print("❌ ERROR: La variable 'models' no está definida.")
    print("📋 Ejecuta la celda de carga de modelos primero")
    raise NameError("Ejecuta la celda de modelos primero")

def run_cumulative_analysis():
    """Ejecutar análisis acumulativo completo."""
    
    print(f"🚀 Iniciando análisis acumulativo")
    print(f"📊 Preguntas: {len(questions)} | Modelos: {len(models)}")
    
    start_time = time.time()
    
    # Resultados por pregunta individual
    individual_results = {}
    
    # Procesar cada pregunta
    for q_idx, question in enumerate(tqdm(questions, desc="Procesando preguntas")):
        question_id = f"q_{q_idx}"
        
        # Textos de pregunta y respuesta
        question_text = f"{question['title']} {question['content']}"
        answer_text = question['accepted_answer']
        
        question_results = {
            'question': question,
            'results': {}
        }
        
        # Evaluar con cada modelo
        for model_key, model in models.items():
            try:
                metrics = simulate_retrieval_and_metrics(
                    question_text, answer_text, model, model_key
                )
                
                question_results['results'][model_key] = {
                    'success': True,
                    'metrics': metrics
                }
                
            except Exception as e:
                question_results['results'][model_key] = {
                    'success': False,
                    'error': str(e),
                    'metrics': {}
                }
        
        individual_results[question_id] = question_results
        
        # Limpiar memoria cada 20 preguntas
        if (q_idx + 1) % 20 == 0:
            torch.cuda.empty_cache()
    
    # Calcular métricas consolidadas
    print("📊 Calculando métricas consolidadas...")
    consolidated_metrics = {}
    
    for model_key in models.keys():
        model_metrics = {}
        
        # Recopilar métricas del modelo
        metrics_data = {}
        
        for result in individual_results.values():
            if model_key in result['results'] and result['results'][model_key]['success']:
                metrics = result['results'][model_key]['metrics']
                
                for metric_name, value in metrics.items():
                    if isinstance(value, (int, float)):
                        if metric_name not in metrics_data:
                            metrics_data[metric_name] = []
                        metrics_data[metric_name].append(value)
        
        # Calcular estadísticas
        for metric_name, values in metrics_data.items():
            if values:
                model_metrics[metric_name] = {
                    'mean': np.mean(values),
                    'std': np.std(values),
                    'median': np.median(values),
                    'min': np.min(values),
                    'max': np.max(values),
                    'count': len(values)
                }
        
        consolidated_metrics[model_key] = model_metrics
    
    # Estadísticas de ejecución
    total_time = time.time() - start_time
    questions_processed = len([r for r in individual_results.values() 
                              if any(result['success'] for result in r['results'].values())])
    
    execution_stats = {
        'questions_processed': questions_processed,
        'questions_failed': len(questions) - questions_processed,
        'total_time': total_time,
        'avg_time_per_question': total_time / len(questions) if questions else 0,
        'success_rate': questions_processed / len(questions) if questions else 0,
        'models_evaluated': len(models),
        'completed_at': datetime.now().isoformat()
    }
    
    results = {
        'individual_results': individual_results,
        'consolidated_metrics': consolidated_metrics,
        'execution_stats': execution_stats,
        'config': config
    }
    
    print(f"✅ Análisis completado en {total_time:.1f}s")
    print(f"📊 Preguntas procesadas: {questions_processed}/{len(questions)}")
    print(f"📈 Tasa de éxito: {questions_processed/len(questions)*100:.1f}%")
    
    return results

# Ejecutar análisis si todo está listo
if config and questions and models:
    print("🎯 Ejecutando análisis acumulativo...")
    results = run_cumulative_analysis()
    
    # Mostrar resumen de resultados
    print("\n🏆 Resumen de resultados por modelo:")
    for model, metrics in results['consolidated_metrics'].items():
        if 'composite_score' in metrics:
            score = metrics['composite_score']['mean']
            std = metrics['composite_score']['std']
            print(f"  {model.upper()}: Score Final = {score:.3f}±{std:.3f}")
    
    print("\n✅ Análisis completado - Listo para guardar resultados")
else:
    print("❌ No se puede ejecutar análisis - faltan componentes")
    print("📋 Asegúrate de ejecutar todas las celdas anteriores en orden")

In [ ]:
# 📊 Funciones de evaluación

def calculate_jaccard_similarity(set1: set, set2: set) -> float:
    """Calcular similitud de Jaccard."""
    if not set1 and not set2:
        return 1.0
    if not set1 or not set2:
        return 0.0
    
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0.0

def calculate_ndcg_at_k(relevant_docs, retrieved_docs, k: int = 10) -> float:
    """Calcular nDCG@K."""
    if not relevant_docs or not retrieved_docs:
        return 0.0
    
    retrieved_k = retrieved_docs[:k]
    true_relevance = [1 if doc in relevant_docs else 0 for doc in retrieved_k]
    
    if sum(true_relevance) == 0:
        return 0.0
    
    try:
        return ndcg_score([true_relevance], [true_relevance])
    except:
        return 0.0

def calculate_precision_at_k(relevant_docs, retrieved_docs, k: int = 5) -> float:
    """Calcular Precision@K."""
    if not relevant_docs or not retrieved_docs:
        return 0.0
        
    retrieved_k = retrieved_docs[:k]
    relevant_retrieved = sum(1 for doc in retrieved_k if doc in relevant_docs)
    
    return relevant_retrieved / len(retrieved_k) if retrieved_k else 0.0

def simulate_retrieval_and_metrics(
    question_text: str, 
    answer_text: str, 
    model,
    model_key: str
):
    """Simular recuperación y calcular métricas."""
    
    # Generar embeddings
    question_embedding = model.encode([question_text])[0]
    answer_embedding = model.encode([answer_text])[0]
    
    # Simular corpus de documentos
    np.random.seed(hash(question_text + model_key) % 2**32)
    
    # Simular documentos recuperados por pregunta y respuesta
    num_docs = 50
    question_docs = [f"q_doc_{i}" for i in range(num_docs)]
    answer_docs = [f"a_doc_{i}" for i in range(num_docs)]
    
    # Simular overlap realista (15-30%)
    overlap_ratio = np.random.uniform(0.15, 0.30)
    overlap_count = int(num_docs * overlap_ratio)
    
    # Crear documentos comunes
    common_docs = [f"common_doc_{i}" for i in range(overlap_count)]
    
    # Mezclar documentos
    question_docs[:overlap_count] = common_docs
    answer_docs[:overlap_count] = common_docs
    
    # Shuffle para simular ranking diferente
    np.random.shuffle(question_docs)
    np.random.shuffle(answer_docs)
    
    # Calcular métricas IR tradicionales
    question_set = set(question_docs[:10])
    answer_set = set(answer_docs[:10])
    
    metrics = {
        'jaccard_similarity': calculate_jaccard_similarity(question_set, answer_set),
        'ndcg_at_10': calculate_ndcg_at_k(answer_docs, question_docs, k=10),
        'precision_at_5': calculate_precision_at_k(answer_docs, question_docs, k=5),
        'common_docs': len(question_set.intersection(answer_set))
    }
    
    # Simular métricas RAG básicas
    metrics.update({
        'faithfulness': np.random.uniform(0.4, 0.9),
        'answer_relevance': np.random.uniform(0.3, 0.8),
        'answer_correctness': np.random.uniform(0.5, 0.9),
        'answer_similarity': np.random.uniform(0.4, 0.8)
    })
    
    # Simular evaluación de calidad LLM
    metrics.update({
        'question_quality': min(0.9, max(0.1, len(question_text) / 200)),
        'answer_quality': min(0.9, max(0.1, len(answer_text) / 500)),
    })
    
    metrics['avg_quality'] = (metrics['question_quality'] + metrics['answer_quality']) / 2
    
    # Score compuesto
    weights = {
        'jaccard_similarity': 0.15,
        'ndcg_at_10': 0.20,
        'precision_at_5': 0.15,
        'faithfulness': 0.15,
        'answer_relevance': 0.15,
        'answer_correctness': 0.10,
        'avg_quality': 0.10
    }
    
    composite_score = sum(
        metrics.get(metric, 0) * weight 
        for metric, weight in weights.items()
    )
    
    metrics['composite_score'] = composite_score
    
    return metrics

print("📊 Funciones de evaluación listas")

In [ ]:
# 🤖 Cargar modelos de embedding

# Mapeo de modelos
MODEL_MAPPING = {
    "mpnet": "multi-qa-mpnet-base-dot-v1",
    "minilm": "all-MiniLM-L6-v2",
    "ada": "all-MiniLM-L6-v2",  # Substituto local
    "e5-large": "intfloat/e5-large-v2"
}

def load_embedding_models(model_keys):
    """Cargar modelos de embedding."""
    models = {}
    print(f"🔄 Cargando modelos en {device}...")
    
    for model_key in model_keys:
        try:
            model_name = MODEL_MAPPING.get(model_key, model_key)
            models[model_key] = SentenceTransformer(model_name, device=device)
            print(f"   ✅ {model_key}")
        except Exception as e:
            print(f"   ⚠️ Error {model_key}: {e}")
            # Fallback a modelo pequeño
            models[model_key] = SentenceTransformer('all-MiniLM-L6-v2', device=device)
            print(f"   ✅ {model_key} (substituto)")
    
    return models

# Cargar modelos según configuración
if config and questions:
    model_keys = list(config['model_config']['embedding_models'].keys())
    models = load_embedding_models(model_keys)
    print(f"✅ {len(models)} modelos listos para evaluación")
else:
    print("❌ No se pueden cargar modelos sin configuración y preguntas")

In [ ]:
# 🗃️ Conectar a ChromaDB y extraer preguntas

def connect_to_chromadb(persist_directory: str = "/content/chromadb"):
    """Conectar a ChromaDB."""
    print(f"🗃️ Conectando a ChromaDB...")
    
    try:
        client = chromadb.PersistentClient(
            path=persist_directory,
            settings=Settings(allow_reset=True)
        )
        print(f"✅ ChromaDB conectado exitosamente")
        return client
        
    except Exception as e:
        print(f"❌ Error conectando a ChromaDB: {e}")
        raise

def get_random_questions_from_chromadb(
    client, 
    collection_name: str, 
    num_questions: int,
    seed: int = 42
):
    """Extraer preguntas aleatorias desde ChromaDB."""
    print(f"🎲 Extrayendo {num_questions} preguntas aleatorias...")
    
    try:
        collection = client.get_collection(name=collection_name)
        total_docs = collection.count()
        print(f"📊 Total de documentos en colección: {total_docs}")
        
        if total_docs < num_questions:
            print(f"⚠️ Solo hay {total_docs} documentos, ajustando cantidad...")
            num_questions = total_docs
        
        # Generar IDs aleatorios
        random.seed(seed)
        all_results = collection.get()
        all_ids = all_results['ids']
        
        selected_ids = random.sample(all_ids, num_questions)
        selected_results = collection.get(ids=selected_ids)
        
        # Procesar resultados
        questions = []
        for i in range(len(selected_results['ids'])):
            metadata = selected_results['metadatas'][i]
            document = selected_results['documents'][i]
            
            question = {
                'id': selected_results['ids'][i],
                'title': metadata.get('title', 'Sin título'),
                'content': metadata.get('question', document[:500] + '...'),
                'accepted_answer': metadata.get('accepted_answer', 'Sin respuesta'),
                'tags': metadata.get('tags', []),
                'metadata': metadata
            }
            questions.append(question)
        
        print(f"✅ {len(questions)} preguntas extraídas exitosamente")
        return questions
        
    except Exception as e:
        print(f"❌ Error extrayendo preguntas: {e}")
        return []

# Conectar y extraer preguntas
if config:
    chromadb_client = connect_to_chromadb()
    num_questions = config['data_config']['num_questions']
    collection_name = "stackoverflow_qa"
    
    questions = get_random_questions_from_chromadb(
        chromadb_client, collection_name, num_questions, seed=42
    )
    
    if questions:
        print(f"✅ {len(questions)} preguntas listas para evaluación")
        
        # Mostrar muestra
        print("\n📋 Muestra de preguntas:")
        for i, q in enumerate(questions[:3]):
            print(f"  {i+1}. {q['title'][:60]}...")
        
        if len(questions) > 3:
            print(f"  ... y {len(questions)-3} preguntas más")
    else:
        print("❌ No se pudieron extraer preguntas")
        raise ValueError("No questions extracted")
else:
    print("❌ Configuración no disponible")

In [ ]:
# 📥 Cargar archivo de preguntas desde configuración

def load_questions_from_config(config):
    """Cargar preguntas desde la configuración."""
    print("📥 Extrayendo preguntas desde la configuración...")
    
    # Verificar si la configuración tiene preguntas incluidas
    if 'questions_data' in config:
        questions_data = config['questions_data']
        print(f"✅ Encontradas {len(questions_data)} preguntas en la configuración")
        
        # Procesar preguntas al formato esperado
        questions = []
        for i, qa_item in enumerate(questions_data):
            question = {
                'id': f"config_q_{i}",
                'title': qa_item.get('title', 'Sin título'),
                'content': qa_item.get('question_content', qa_item.get('question', '')),
                'accepted_answer': qa_item.get('accepted_answer', 'Sin respuesta'),
                'ms_links': qa_item.get('ms_links', []),
                'tags': qa_item.get('tags', []),
                'metadata': qa_item
            }
            questions.append(question)
        
        return questions
    
    # Si no hay preguntas en la config, intentar cargar desde Drive
    elif 'questions_file' in config:
        questions_filename = config['questions_file']
        print(f"📥 Intentando cargar preguntas desde: {questions_filename}")
        
        file_id = get_drive_file_by_name(questions_filename)
        if not file_id:
            print(f"❌ No se encontró el archivo: {questions_filename}")
            return []
        
        try:
            request = service.files().get_media(fileId=file_id)
            file_io = io.BytesIO()
            downloader = MediaIoBaseDownload(file_io, request)
            
            done = False
            while done is False:
                status, done = downloader.next_chunk()
            
            file_io.seek(0)
            questions_data = json.loads(file_io.read().decode('utf-8'))
            
            if isinstance(questions_data, list):
                questions = []
                for i, qa_item in enumerate(questions_data):
                    question = {
                        'id': f"file_q_{i}",
                        'title': qa_item.get('title', 'Sin título'),
                        'content': qa_item.get('question_content', qa_item.get('question', '')),
                        'accepted_answer': qa_item.get('accepted_answer', 'Sin respuesta'),
                        'ms_links': qa_item.get('ms_links', []),
                        'tags': qa_item.get('tags', []),
                        'metadata': qa_item
                    }
                    questions.append(question)
                
                print(f"✅ {len(questions)} preguntas cargadas desde archivo")
                return questions
            else:
                print("❌ Formato de archivo de preguntas no válido")
                return []
                
        except Exception as e:
            print(f"❌ Error cargando archivo de preguntas: {e}")
            return []
    
    else:
        print("❌ No se encontraron preguntas en la configuración")
        print("💡 La configuración debe incluir 'questions_data' o 'questions_file'")
        return []

def download_embeddings_from_drive():
    """Descargar archivos de embeddings desde Google Drive."""
    print("📥 Preparando descarga de archivos de embeddings...")
    
    # Crear directorio local para embeddings
    embeddings_dir = "/content/embeddings_data"
    import os
    os.makedirs(embeddings_dir, exist_ok=True)
    
    # Archivos de embedding esperados
    embedding_files = {
        'ada': 'docs_ada_with_embeddings_20250721_123712.parquet',
        'e5-large': 'docs_e5large_with_embeddings_20250721_124918.parquet', 
        'mpnet': 'docs_mpnet_with_embeddings_20250721_125254.parquet',
        'minilm': 'docs_minilm_with_embeddings_20250721_125846.parquet'
    }
    
    downloaded_files = {}
    
    for model_key, filename in embedding_files.items():
        local_path = os.path.join(embeddings_dir, filename)
        
        # Verificar si el archivo ya existe localmente
        if os.path.exists(local_path):
            # Verificar tamaño del archivo
            file_size = os.path.getsize(local_path)
            file_size_mb = file_size / (1024 * 1024)
            
            if file_size_mb > 10:  # Si el archivo tiene más de 10MB, asumimos que está completo
                print(f"✅ {model_key}: {filename} ya existe ({file_size_mb:.1f} MB) - omitiendo descarga")
                downloaded_files[model_key] = local_path
                continue
            else:
                print(f"⚠️ {model_key}: archivo existe pero es muy pequeño ({file_size_mb:.1f} MB) - re-descargando")
                os.remove(local_path)
        
        # Buscar y descargar archivo desde Drive
        print(f"🔍 Buscando: {filename}")
        
        file_id = get_drive_file_by_name(filename)
        if not file_id:
            print(f"⚠️ No encontrado: {filename}")
            continue
        
        try:
            print(f"📥 Descargando {model_key}: {filename}")
            request = service.files().get_media(fileId=file_id)
            with open(local_path, 'wb') as f:
                downloader = MediaIoBaseDownload(f, request)
                done = False
                while done is False:
                    status, done = downloader.next_chunk()
                    if status:
                        progress = int(status.progress() * 100)
                        print(f"   📥 {model_key}: {progress}%", end='\r')
            
            # Verificar el archivo descargado
            file_size = os.path.getsize(local_path)
            file_size_mb = file_size / (1024 * 1024)
            downloaded_files[model_key] = local_path
            print(f"   ✅ {model_key}: {filename} ({file_size_mb:.1f} MB)")
            
        except Exception as e:
            print(f"   ❌ Error descargando {filename}: {e}")
    
    return downloaded_files

# Cargar preguntas según configuración
if config:
    questions = load_questions_from_config(config)
    
    if questions:
        print(f"✅ {len(questions)} preguntas listas para evaluación")
        
        # Mostrar muestra
        print("\n📋 Muestra de preguntas:")
        for i, q in enumerate(questions[:3]):
            print(f"  {i+1}. {q['title'][:60]}...")
            print(f"      Enlaces MS: {len(q['ms_links'])}")
        
        if len(questions) > 3:
            print(f"  ... y {len(questions)-3} preguntas más")
    else:
        print("❌ No se pudieron cargar preguntas")
        raise ValueError("No questions loaded from configuration")
    
    # Descargar archivos de embeddings (con verificación de existencia)
    print("\n📦 Verificando archivos de embeddings...")
    embedding_files = download_embeddings_from_drive()
    
    if not embedding_files:
        print("❌ No se pudieron obtener archivos de embeddings")
        raise ValueError("No embedding files available")
    else:
        print(f"\n✅ {len(embedding_files)} archivos de embeddings disponibles")
        
        # Mostrar resumen de archivos
        total_size = 0
        for model_key, file_path in embedding_files.items():
            if os.path.exists(file_path):
                size_mb = os.path.getsize(file_path) / (1024 * 1024)
                total_size += size_mb
                print(f"  📊 {model_key}: {size_mb:.1f} MB")
        
        print(f"  📦 Total: {total_size:.1f} MB")
        
else:
    print("❌ Configuración no disponible")

In [ ]:
# 🔐 Autenticación con Google Drive
print("🔐 Configurando acceso a Google Drive...")

# Autenticar y montar Google Drive
auth.authenticate_user()
drive.mount('/content/drive')

# Configurar servicio de Google Drive API
service = build('drive', 'v3')

print("✅ Google Drive configurado correctamente")
print("📂 Drive montado en: /content/drive")

In [ ]:
# 📚 Importaciones y configuración inicial
import json
import time
import random
import numpy as np
import pandas as pd
from datetime import datetime
from typing import Dict, List, Any, Optional
import warnings
warnings.filterwarnings('ignore')

# Google Drive
from google.colab import auth, drive
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload, MediaIoBaseDownload
import io

# ML y NLP
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score

# ChromaDB
import chromadb
from chromadb.config import Settings

# Progress tracking
from tqdm import tqdm

# Verificar GPU
gpu_available = torch.cuda.is_available()
device = torch.device('cuda' if gpu_available else 'cpu')

print("📚 Importaciones completadas")
print(f"🚀 GPU disponible: {gpu_available}")
if gpu_available:
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Memoria: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"💻 Dispositivo: {device}")

In [ ]:
# 📦 Instalación de dependencias
print("📦 Instalando dependencias...")

# Instalar paquetes necesarios
!pip install -q chromadb sentence-transformers numpy pandas scikit-learn tqdm
!pip install -q google-api-python-client google-auth-oauthlib google-auth-httplib2
!pip install -q torch

print("✅ Dependencias instaladas correctamente")

# 📊 Análisis Acumulativo de N Preguntas - Google Colab

Este notebook implementa el análisis acumulativo de múltiples preguntas comparando la efectividad de diferentes modelos de embedding en la recuperación de documentos usando preguntas vs respuestas.

## 🎯 Objetivos
- Evaluar múltiples modelos de embedding simultáneamente
- Comparar recuperación usando preguntas vs respuestas aceptadas
- Calcular métricas IR tradicionales, RAG y calidad LLM
- Generar análisis estadístico completo de los resultados

## 📋 Flujo de Trabajo
1. **Configuración**: Cargar configuración desde Google Drive
2. **Datos**: Extraer preguntas aleatorias desde ChromaDB
3. **Evaluación**: Ejecutar análisis con GPU para cada modelo
4. **Resultados**: Guardar métricas consolidadas y detalladas
5. **Visualización**: Ver resultados en Streamlit

## 📋 Instrucciones:

1. **Activar GPU**: Runtime → Change runtime type → GPU → T4
2. **Configurar archivo**: Cambiar CONFIG_FILENAME en la celda de configuración
3. **Ejecutar todo**: Runtime → Run all (Ctrl+F9)
4. **Autorizar Drive**: Cuando se solicite acceso a Google Drive
5. **Monitorear progreso**: Ver barras de progreso

## ✨ Características:
- 🚀 **Aceleración GPU** para procesamiento rápido
- 📊 **Métricas completas**: IR tradicionales, RAG y calidad LLM
- 🔍 **Comparación pregunta vs respuesta**
- 📈 **Resultados automáticos** guardados en Google Drive

## 📤 Resultados:
- Se guardan automáticamente en Google Drive
- Vuelve a Streamlit para ver visualizaciones
- Ve a "Resultados Análisis N Preguntas"